In [ ]:
import numpy as np
from collections import defaultdict
import random
from tqdm import trange
import copy
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import sys
import igraph
from matplotlib import cm, colors
random.seed(42)

In [ ]:
def rargmax(x):
    # return x.argmax()  # <- favors clustering towards smaller state numbers
    return np.random.choice((x == x.max()).nonzero()[0])

In [ ]:
def datagen_structured_obs_room(
    room,
    start_r=None,
    start_c=None,
    no_left=[],
    no_right=[],
    no_up=[],
    no_down=[],
    length=10000,
    seed=42,
):
    """room is a 2d numpy array. inaccessible locations are marked by -1.
    start_r, start_c: starting locations

    In addition, there are invisible obstructions in the room
    which disallows certain actions from certain states.

    no_left:
    no_right:
    no_up:
    no_down:

    Each of the above are list of states from which the corresponding action is not allowed.

    """
    np.random.seed(seed)
    H, W = room.shape
    if start_r is None or start_c is None:
        start_r, start_c = np.random.randint(H), np.random.randint(W)

    actions = np.zeros(length, int)
    x = np.zeros(length, int)  # observations
    rc = np.zeros((length, 2), int)  # actual r&c

    r, c = start_r, start_c
    x[0] = room[r, c]
    rc[0] = r, c

    count = 0
    while count < length - 1:

        act_list = [0, 1, 2, 3]  # 0: left, 1: right, 2: up, 3: down
        if (r, c) in no_left:
            act_list.remove(0)
        if (r, c) in no_right:
            act_list.remove(1)
        if (r, c) in no_up:
            act_list.remove(2)
        if (r, c) in no_down:
            act_list.remove(3)

        a = np.random.choice(act_list)

        # Check for actions taking out of the matrix boundary.
        prev_r = r
        prev_c = c
        if a == 0 and 0 < c:
            c -= 1
        elif a == 1 and c < W - 1:
            c += 1
        elif a == 2 and 0 < r:
            r -= 1
        elif a == 3 and r < H - 1:
            r += 1

        # Check whether action is taking to inaccessible states.
        temp_x = room[r, c]
        if temp_x == -1:
            r = prev_r
            c = prev_c
            pass

        actions[count] = a
        x[count + 1] = room[r, c]
        rc[count + 1] = r, c
        count += 1

    return actions, x, rc


In [ ]:
room = np.array(
    [
        [1, 2, 3, 0, 3, 1, 1, 1],
        [1, 1, 3, 2, 3, 2, 3, 1],
        [1, 1, 2, 0, 1, 2, 1, 0],
        [0, 2, 1, 1, 3, 0, 0, 2],
        [3, 3, 1, 0, 1, 0, 3, 0],
        [2, 1, 2, 3, 3, 3, 2, 0],
    ]
)
actions, observations, rc = datagen_structured_obs_room(room, length=50)     #Use length=50000 for bigger room

In [ ]:
# Count occurrences of each unique value
unique, counts = np.unique(room, return_counts=True)
# Create a dictionary of the counts
result = dict(zip(unique, counts))

print("Counts of 0, 1, 2, 3:")
print(result)

In [ ]:
a = [1,2,3,4,5]
c = []
c = np.concatenate((c,a))
c.reshape(-1,1)

In [ ]:
# @nb.njit
class Agent:
    def __init__(self, num_obs=4, num_actions=4, pseudocount=1e-3):
        self.num_obs = num_obs
        self.groups_of_tables = {}
        self.table_totals = {}  # Keep track of totals for each table separately
        self.total_observations = 0  # Keep track of total observations across all tables
        self.count = np.ones((num_obs,num_obs,num_actions))
        self.TM = self.count.copy()
        self.C = np.zeros((num_obs,num_obs,num_actions))
        self.normalize_TM()
        self.initialize_clones(num_obs)
        self.pseudocount = pseudocount


    def initialize_clones(self,num_obs):
        for restaurant_id in range(num_obs):
            self.groups_of_tables[restaurant_id] = {}
            self.groups_of_tables[restaurant_id][0] = 1
    
    def normalize_TM(self):
        num_obs = np.shape(self.count)[0]
        num_actions = np.shape(self.count)[2]
        count = self.count.copy()
        for s in range(num_obs):
            for a in range(num_actions):
                self.TM[s, :, a] = count[s,:,a] / count[s,:,a].sum()
                
                
    def add_clone(self, restaurant_id, table_id):
        """Add exactly one clone to a specified table, creating the table or group if necessary."""
        # Automatically create the group and table if they don't exist
        if restaurant_id not in self.groups_of_tables:
            self.groups_of_tables[restaurant_id] = {}
        if table_id not in self.groups_of_tables[restaurant_id]:
            self.groups_of_tables[restaurant_id][table_id] = 0  # Initialize clones count for the table

        # Add one clone to the table count and update total observations
        self.groups_of_tables[restaurant_id][table_id] += 1
        self.table_totals[(restaurant_id, table_id)] = self.groups_of_tables[restaurant_id][table_id]  # Update table total
        self.total_observations += 1

    def get_total_observations(self):
        """Return the total number of observations."""
        return self.total_observations

    def get_restaurant_total_customers(self, restaurant_id):
        """Return the total number of clones in all tables within a specific restaurant."""
        return sum(self.groups_of_tables.get(restaurant_id, {}).values())

    def get_table_total_customers(self, restaurant_id, table_id):
        """Return the total number of clones for a specific table."""
        return self.groups_of_tables.get(restaurant_id, {}).get(table_id, 0)
    
#     def get_prior_distribution(self):

#         return [self.get_restaurant_total_customers(restaurant_id for restaurant_id in range(self.num_obs))
#                 /self.get_total_observations()]

    def count_tables_in_restaurant(self, restaurant_id):
        """Returns the number of tables within the specified restaurant."""
        if restaurant_id in self.groups_of_tables:
            return len(self.groups_of_tables[restaurant_id])
        else:
            # print(f"Group {group_id} does not exist.")
            return 0
        
# agent.update_count((total_clones, prev_action, obs_ind))
    def expand_split(self):
        # expand dimension
        orig_count = self.count.copy()
        orig_TM = self.TM.copy()
        orig_C = self.C.copy()
        n_prev_clones = np.shape(orig_count)[0]
        n_actions = np.shape(orig_count)[2]
        expanded_count = np.zeros((n_prev_clones+1, n_prev_clones+1, n_actions)) + self.pseudocount
        expanded_TM = np.zeros((n_prev_clones+1, n_prev_clones+1, n_actions)) + self.pseudocount
        expanded_C = np.zeros((n_prev_clones+1, n_prev_clones+1, n_actions)) + self.pseudocount
                
        # Copy the original matrix values into the top-left submatrix of the expanded matrix
        expanded_count[:n_prev_clones, :n_prev_clones, :] = orig_count
        expanded_TM[:n_prev_clones, :n_prev_clones, :] = orig_TM
        expanded_C[:n_prev_clones, :n_prev_clones, :] = orig_C
        
        self.count = expanded_count
        self.TM = expanded_TM
        self.C = expanded_C
        
# #         self.count[state, state2]
#         for ss in state2:
#             self.count[state, ss, action] += 1
            
    def update_count(self, state, state2, action): # updating counts when not splitted
#         for ss in state2: 
#         self.count[state, state2]
#         for ss in state2:
        self.count[state, state2, action] += 1

        
#         for next_state in state2: 
            
#     def update_transition(self, state, state2, action): 
#         # update transition probability p(s'|s,a)
#         self.TM[state, state2, action] += 1
        
#         # Normalize the probabilities for the current state and action
#         self.TM[state, :, action] /= self.TM[state, :, action].sum()        
    def merged_likelihood(self, clone_map):
        
        merged_TM = self.TM.copy()
        
        curr_obs, clone_num, unique_idx = random.choice(clone_map)
        shared_obs_clones = [t for t in clone_map if t[0]==curr_obs and t[1] != clone_num] # all the clones that share the obs
        
        compare_clones = random.choice(shared_obs_clones) # pick a random clone that shares observation 

        state_a = unique_idx
        state_b = compare_clones[2]
        
        for s in range(len(clone_map)):
            if s != state_a and s != state_b:                
                merged_TM[state_a][s] += merged_TM[state_b][s]
                merged_TM[s][state_a] += merged_TM[s][state_b] 
                # Remove state_b by zeroing out probabilities (optional)
                merged_TM[state_b][s] = 0
                merged_TM[s][state_b] = 0
                
# agent.update_C(mess_fwd, mess_bwd, observations[:o+1], actions[:o+1], clone_map)
    def update_C(self, mess_fwd, mess_bwd, observations, actions, clone_map, clone_history):
        timesteps = len(observations)
        reversed_mess_bwd = mess_bwd[::-1]
        for t in range(timesteps-1):
            aij, i, j = (
                actions[t],
                observations[t],
                observations[t+1],
            )  # at time t-1 -> t we go from observation i to observation j
            fwd = mess_fwd[t]
            bwd = reversed_mess_bwd[t+1]

            current_clones = [clone[2] for clone in clone_map 
                             if clone[0]==clone_history[t+1][0]]
            previous_clones = [clone[2] for clone in clone_map 
                             if clone[0]==clone_history[t][0]]
#             print("current clones: {}".format(current_clones))
#             print("previous clones: {}".format(previous_clones))
            for c, curr in enumerate(current_clones):
                for p, prev in enumerate(previous_clones):
#                     print('\n')
#                     print("fwd: {}".format(fwd))
#                     print("bwd: {}".format(bwd))
#                     print("c: {}".format(c))
#                     print("p: {}".format(p))
#                     print("prev: {}".format(prev))
#                     print("curr: {}".format(curr))
                    
                    q = (
                    fwd[p] * self.TM[prev, curr, aij] * bwd[c])
                    self.C[prev, curr, aij] += q
#         prev_action = actions[t-1]
#         prev_message = message
#         message = np.zeros(len(current_clones))
#         for c, curr in enumerate(current_clones): # loop over indices for currently activated clones
#             message_ = 0
#             for p, prev in enumerate(previous_clones): # loop over indices for previously activated clones           
#                 if prev != curr: 
#                     message_ += (TM[prev, curr, prev_action] * prev_message[p])
#             message[c] = message_
#         p_obs = message.sum()
            
            
            
            
            
            
        
def CRP(agent, curr_observation, alpha=1.0, beta=1.0):
    """
    Simulates the Chinese Restaurant Process.

    Parameters:
    - history: int, the total number of customers to simulate.
    - alpha: float, the concentration parameter.

    Returns:
    - A list where the i-th element represents the table number of the i-th customer.
    """

    n = agent.get_restaurant_total_customers(curr_observation)


    if curr_observation not in agent.groups_of_tables:
        agent.add_clone(curr_observation,0)
        table_choice = 0
        assignments = 0
        probs = 1
    else:
        
        probs = [clone_count / (n + alpha) for table_id, clone_count in 
               agent.groups_of_tables[curr_observation].items()] + [alpha / (n + alpha)] # This is the prior
#         print(probs)
    
#     probs = np.concatenate(probs)
#     print(probs)
    # Add an update rule (ref Nora's paper 1st equation)

    # Choose a table based on the probabilities
    table_choice = np.random.choice(len(probs), p=probs)


    # update clone --> existing or new, same
    agent.add_clone(curr_observation,table_choice)
    assignments = table_choice

    return assignments, probs


def get_distribution(data):
    """
    Compute the probability distribution from a nested dictionary structure.

    Parameters:
        data (dict): A nested dictionary where the outer keys map to inner dictionaries
                     containing counts for different elements.

    Returns:
        np.ndarray: A NumPy array containing the probability distribution.
    """
    # Flatten the dictionary and compute total count
    flattened_counts = defaultdict(int)
    total_count = 0
    ct = 0
    for outer_key, inner_dict in data.items():
#         print(outer_key)
        for inner_key, count in inner_dict.items():
#             print(inner_key)
            flattened_counts[ct] += count
            total_count += count
            ct += 1

    # Compute probabilities as a sorted NumPy array
    sorted_keys = sorted(flattened_counts.keys())
    probabilities = np.array([flattened_counts[key] / total_count for key in sorted_keys])

    return probabilities



def forward_algorithm(agent, prior, observations, actions, clone_map, clone_history, TM, store_messages=False):
    num_steps = len(observations)
    num_states = np.shape(TM)[0]
    log_forward_probs = []
    log2_lik = np.zeros(num_steps)
    previous_clones = []
    initial_state = observations[0]
    mess_fwd = []
    for c, clone in enumerate(clone_map):
        if clone[0] == initial_state: # if this observation
            log_forward_probs.append(prior[c])
            previous_clones.append(clone[2])
    message = log_forward_probs
    p_obs = np.sum(message)   
    assert p_obs > 0
    message /= p_obs
    log2_lik[0] = np.log2(p_obs)
    if store_messages: 
        mess_fwd.append(message)
    for t in range(1, num_steps):        
        # 2024-11-26: following the cscg-orig code, calculate the likelihood based on obs (not clone)
        current_clones = [clone[2] for clone in clone_map 
                         if clone[0]==clone_history[t][0]]
        prev_action = actions[t-1]
        prev_message = message
        message = np.zeros(len(current_clones))
        for c, curr in enumerate(current_clones): # loop over indices for currently activated clones
            message_ = 0
            for p, prev in enumerate(previous_clones): # loop over indices for previously activated clones           
                if prev != curr: 
                    message_ += (TM[prev, curr, prev_action] * prev_message[p])
            message[c] = message_
        p_obs = message.sum()
        assert p_obs > 0
        message /= p_obs
        log2_lik[t] = np.log2(p_obs)        
        if store_messages:
            mess_fwd.append(message)        
        previous_clones = current_clones
    return log2_lik, mess_fwd

def forward_mp(agent, prior, observations, actions, clone_map, clone_history, TM, store_messages=False):
    num_steps = len(observations)
    num_states = np.shape(TM)[0]
    log_forward_probs = []
    log2_lik = np.zeros(num_steps)
    previous_clones = []
    initial_state = observations[0]
    mess_fwd = []
    for c, clone in enumerate(clone_map):
        if clone[0] == initial_state: # if this observation
            log_forward_probs.append(prior[c])
            previous_clones.append(clone[2])
    message = log_forward_probs
    p_obs = np.sum(message)   
    assert p_obs > 0
    message /= p_obs
    log2_lik[0] = np.log2(p_obs)
    if store_messages: 
        mess_fwd.append(message)
    for t in range(1, num_steps):        
        # 2024-11-26: following the cscg-orig code, calculate the likelihood based on obs (not clone)
        current_clones = [clone[2] for clone in clone_map 
                         if clone[0]==clone_history[t][0]]
        prev_action = actions[t-1]
        prev_message = message
#         message = np.zeros(len(current_clones))
        new_message = np.zeros(len(current_clones))
        for c, curr in enumerate(current_clones): # loop over indices for currently activated clones
#             message_ = 0
            new_message_ = []
            for p, prev in enumerate(previous_clones): # loop over indices for previously activated clones           
                if prev != curr: 
#                     message_ += (TM[prev, curr, prev_action] * prev_message[p])
                    new_message_.append(TM[prev, curr, prev_action] * prev_message[p])
#             message[c] = message_
            new_message[c] = max(new_message_)
        message = new_message
        p_obs = message.max()
        assert p_obs > 0
        message /= p_obs
        log2_lik[t] = np.log2(p_obs)        
        if store_messages:
            mess_fwd.append(message)        
        previous_clones = current_clones
    return log2_lik, mess_fwd

def backward_algorithm(agent, observations, actions, clone_map, TM):
    mess_bwd = []
    t = observations.shape[0] - 1
#     i = observations[t]
    current_clones = [clone[2] for clone in clone_map 
                     if clone[0]==clone_history[t][0]]    
    message = np.ones(len(current_clones)) / len(current_clones)
    message /= message.sum()
    mess_bwd.append(message)
    for t in range(observations.shape[0]-2, -1, -1):
        previous_clones = [clone[2] for clone in clone_map 
                 if clone[0]==clone_history[t][0]]
        prev_action = actions[t]
        prev_message = message
        
        message = np.zeros(len(previous_clones))
            
        for p, prev in enumerate(previous_clones):
            message_ = 0
            for c, curr in enumerate(current_clones):
                if prev != curr: 
                    message_ += (TM[prev, curr, prev_action] * prev_message[c])
            message[p] = message_
        p_obs = message.sum()
        assert p_obs > 0
        message /= p_obs
        mess_bwd.append(message)
        
        current_clones = previous_clones
        
    return mess_bwd

# states = backtrace(agent, observations, actions, mess_fwd)
def backtrace(agent, observations, actions, mess_fwd, clone_map):
    """Computes backward messages"""
    code = np.zeros(observations.shape[0], dtype=np.int64)
    TM = agent.TM.copy()
    # backward pass
    t = observations.shape[0] - 1
    i = observations[t]
#     t_start, t_stop = mess_loc[t : t + 2]
#     rev_mess_fwd = mess_fwd[::-1]
#     belief = rev_mess_fwd[0]
    belief = mess_fwd[t]
    curr_clone = rargmax(belief)
    
    
# [clone[2] for clone in clone_map 
#                  if clone[0]==clone_history[t][0]]    
    for clone in clone_map: 
        if clone[0] == i and clone[1] == curr_clone:
            code[t] = clone[2]
#     code[t] = [clone[2] for clone in clone_map if clone[0]==i and clone[1]==curr_clone]
    
#     code[t] = rargmax(belief) # this is which clone
#     print(t)
    for t in range(observations.shape[0] - 2, 1, -1):
        print(t)
        prev_action, prev_obs, curr_obs = (
            actions[t-1],
            observations[t-1],
            observations[t],
        )  # at time t -> t+1 we go from observation i to observation j
#         (i_start, i_stop), j_start = state_loc[i : i + 2], state_loc[j]
#         t_start, t_stop = mess_loc[t : t + 2]
        previous_clones = [clone[2] for clone in clone_map 
                 if clone[0]==prev_obs]
        current_clones = [clone[2] for clone in clone_map 
             if clone[0]==curr_obs]
        belief = []

#         print(prev_obs)
        print(len(mess_fwd[t]))
        print(len(current_clones))
        print(len(previous_clones))
#         print(observations[t])
#         print(previous_clones)
#         print(code[t+1])
#         print('previous clones'.format(previous_clones))
#         print('code'.format(code[t+1]))
        for p, prev in enumerate(previous_clones):
            belief.append(mess_fwd[t][p] * TM[prev, code[t+1], prev_action])
#         belief = (
#             mess_fwd[t] * T[aij, obser, code[t+1]]
#         )
        curr_clone = rargmax(np.array(belief))
        for clone in clone_map: 
            if clone[0] == prev_obs and clone[1] == curr_clone:
                code[t] = clone[2]
#     states = state_loc[x] + code
    return code  

def decode(agent, observations, actions, clone_map, clone_history):
    """Compute the MAP assignment of latent variables using max-product message passing."""
    prior = get_distribution(agent.groups_of_tables)    
    TM = agent.TM.copy()
    log2_lik, mess_fwd = forward_mp(agent, prior, observations, actions, 
                                    clone_map, clone_history, TM, store_messages=True)
    states = backtrace(agent, observations, actions, mess_fwd, clone_map)
    
#     log2_lik, mess_fwd = forward_mp(
#         self.T.transpose(0, 2, 1),
#         self.Pi_x,
#         self.n_clones,
#         x,
#         a,
#         store_messages=True,
#     )
#     states = backtrace(self.T, self.n_clones, x, a, mess_fwd)
    return -log2_lik, states

def clean_clone_map(data, delete_items):


    updated_data=[]
    for ids in delete_items: 
        for x,y,uid in data: 
            if uid < ids[2]:
                updated_data.append((x,y,uid))
            elif uid > ids[2]: 
                if x != ids[0]: 
                    updated_data.append((x,y,uid-1))
                else:
                    updated_data.append((x,y-1,uid-1))

    return updated_data

def binarizing_edges(agent):
    TM = agent.TM.copy()
    n_states = np.shape(TM)[0]
    n_actions = np.shape(TM)[2]
    for s1 in range(n_states):
    #         for s2 in range(n_states):
        TM_t = np.transpose(TM[s1])
        for action in range(n_actions):
            currTM = TM_t[action]

            for e,element in enumerate(currTM): 
                if element == np.max(currTM):
                    TM[s1][e][action]=1
                else:
                    TM[s1][e][action]=0

    return TM



def sigmoid(x):
    """
    Compute the sigmoid of x.
    
    Parameters:
    - x: A scalar, vector, or matrix.
    
    Returns:
    - Sigmoid of x.
    """
    return 1 / (1 + np.exp(-x))

def normalize_TM_count(count):
    num_obs = np.shape(count)[0]
    num_actions = np.shape(count)[2]
    new_count = count.copy()
    for s in range(num_obs):
        for a in range(num_actions):
            new_count[s, :, a] = count[s,:,a] / count[s,:,a].sum()
    return new_count



def draw_graph(agent, clone_map, niter=0, threshold=0.2, binary_edges=True):
    # Example transition matrix (state, state, action)
    # transition_matrix = agent.TM
    if binary_edges:
        TM=binarizing_edges(agent)
    else:         
        TM = agent.TM
    num_states = np.shape(TM)[0]
    num_actions = np.shape(TM)[2]

    # Threshold to show only significant transitions
#     threshold = 0.2

    # Create a directed graph
    G = nx.DiGraph()

    # Add nodes for each state
    for state in range(num_states):
        G.add_node(state)

    # Add edges based on the transition matrix
    for action in range(num_actions):
        for from_state in range(num_states):
            for to_state in range(num_states):
                weight = TM[from_state, to_state, action]
                if weight > threshold:  # Filter out weak transitions
                    G.add_edge(
                        from_state,
                        to_state,
                        weight=weight,
                        label=f"A{action}"  # Label edges by action
                    )

    # Use Kamada-Kawai layout for a more spatially balanced layout
    pos = nx.kamada_kawai_layout(G)  # Alternative layouts: spring_layout, shell_layout

    # Node colors based on clone_map (or any observation data)
    node_colors = [t[0] for t in clone_map]  # Replace with your actual node color logic

    color_map = {0: 'pink', 1: 'skyblue', 2: 'green', 3: 'orange'}  # Observation to color mapping
    node_colors = [color_map[val] for val in node_colors]  # Flatten room to assign colors to each cell

    # Draw the graph
    plt.figure(figsize=(12, 8))
    nx.draw(
        G,
        pos,
        with_labels=True,
        node_color=node_colors,  # Use the color scheme for nodes
        node_size=1000,
        font_size=10,
        edge_color='gray',
        arrowsize=20
    )

    # Draw edge labels
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(
        G,
        pos,
        edge_labels=edge_labels,
        font_size=8
    )

    plt.title("Iteration {}".format(niter))
    plt.axis('off')  # Turn off axes for cleaner visualization
    plt.show()
    # 
    
def draw_graph_TM(agent, clone_map, TM, niter=0, threshold=0.2, binary_edges=True):
    # Example transition matrix (state, state, action)
    # transition_matrix = agent.TM
    # if binary_edges:
    #     TM=binarizing_edges(agent)
    # else:         
    #     TM = agent.TM
    num_states = np.shape(TM)[0]
    num_actions = np.shape(TM)[2]

    # Threshold to show only significant transitions
#     threshold = 0.2

    # Create a directed graph
    G = nx.DiGraph()

    # Add nodes for each state
    for state in range(num_states):
        G.add_node(state)

    # Add edges based on the transition matrix
    for action in range(num_actions):
        for from_state in range(num_states):
            for to_state in range(num_states):
                weight = TM[from_state, to_state, action]
                if weight > threshold:  # Filter out weak transitions
                    G.add_edge(
                        from_state,
                        to_state,
                        weight=weight,
                        label=f"A{action}"  # Label edges by action
                    )

    # Use Kamada-Kawai layout for a more spatially balanced layout
    pos = nx.kamada_kawai_layout(G)  # Alternative layouts: spring_layout, shell_layout

    # Node colors based on clone_map (or any observation data)
    node_colors = [t[0] for t in clone_map]  # Replace with your actual node color logic

    color_map = {0: 'pink', 1: 'skyblue', 2: 'green', 3: 'orange'}  # Observation to color mapping
    node_colors = [color_map[val] for val in node_colors]  # Flatten room to assign colors to each cell

    # Draw the graph
    plt.figure(figsize=(12, 8))
    nx.draw(
        G,
        pos,
        with_labels=True,
        node_color=node_colors,  # Use the color scheme for nodes
        node_size=1000,
        font_size=10,
        edge_color='gray',
        arrowsize=20
    )

    # Draw edge labels
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(
        G,
        pos,
        edge_labels=edge_labels,
        font_size=8
    )

    plt.title("Iteration {}".format(niter))
    plt.axis('off')  # Turn off axes for cleaner visualization
    plt.show()
# def draw_graph_cscg()

In [ ]:
def merge(agent, observations, actions, split_lik, clone_map, clone_history, n_particles=10, ): 
    log_lik = split_lik.copy()
    for particle in range(n_particles): # many hypotheses // can this be parallelized?

        curr_obs, clone_num, unique_idx = random.choice(clone_map[n_obs:])
        shared_obs_clones = [t for t in clone_map[n_obs:] if t[0]==curr_obs and t[1] != clone_num] # all the clones that share the obs
    #     prev_ind = [t[2] for t in clone_map if t[0]==prev_obs]
    #     _, clone_id = clone_list[idx]
        # do the merging by iterating over other clones
        # in the sequence, find other clones that could be merged with the current

        compare_clones = random.choice(shared_obs_clones) # pick a random clone that shares observation 
        #### make this selection anti-proportional to number of assigned observations???

        state_a = unique_idx
        state_b = compare_clones[2]
#         merged_TM = agent.TM.copy()
        merged_count = agent.count.copy()

        # adjusting prior distribution
        merged_prior = get_distribution(agent.groups_of_tables)
        merged_prior[state_a] += merged_prior[state_b]
        merged_prior[state_b] = 0 #pseudocount    
        # adjusting TM
        for s in range(np.shape(agent.TM)[0]):
            if s != state_a and s != state_b:
                for a in range(len(np.unique(actions))):
#                     merged_TM[state_a][s][a] += merged_TM[state_b][s][a]
#                     merged_TM[s][state_a][a] += merged_TM[s][state_b][a]
#                     merged_TM[s][state_b][a] = pseudocount
#                     merged_TM[state_b][s][a] = pseudocount
                    merged_count[state_a][s][a] += merged_count[state_b][s][a]
                    merged_count[s][state_a][a] += merged_count[s][state_b][a]
                    merged_count[s][state_b][a] = pseudocount
                    merged_count[state_b][s][a] = pseudocount
        merged_TM = normalize_TM_count(merged_count)
        # Remove state_b by zeroing out probabilities (optional)
    #     agent.TM[state_b] = np.zeros(self.num_states)    

        # Get the new prior
    #     new_prior = # GET THE NEW DISTRIBUTION

        # Get the new likelihood by forward algorithm
    #     new_lik = new_prior * new_Tm

        new_lik_, mess_fwd = forward_algorithm(agent, merged_prior, observations, actions, clone_map, clone_history, merged_TM)
#         print(len(mess_fwd))
        #     print(log_lik, new_lik)
        new_lik = new_lik_.mean()
        if new_lik > log_lik: # if merging is better: 
            total_lls.append(new_lik)
            total_indices.append(1)
            agent.TM = merged_TM
            agent.groups_of_tables[curr_obs][clone_num] += agent.groups_of_tables[curr_obs][compare_clones[1]]
            agent.groups_of_tables[curr_obs][compare_clones[1]] = 0
            print("Particle {}:".format(particle))
            print("Merged Obs {} clone {} into clone {}, log lik: {}".format(curr_obs, compare_clones[1], clone_num, new_lik))
#             pbar.set_postfix(train_bps=new_lik)

            # clean up clones (including values that are 0)
#             print(agent.groups_of_tables[curr_obs])
            orig_table = agent.groups_of_tables[curr_obs].copy()
#             print(orig_table)
            orig_table[compare_clones[1]] = 0
            delete_indices = [o for o,i in enumerate(orig_table) if orig_table[i]==0]
            # Get unique IDs for the delete_indices from clone_map
            delete_items = [
                t for t in clone_map
                for d in delete_indices
                if t[0] == curr_obs and t[1] == d
            ]
#                     delete_uniqueids = [
#                 t[2] for t in clone_map
#                 for d in delete_indices
#                 if t[0] == curr_obs and t[1] == d
#             ]
            new_table = {}
            ni = 0
            for i in range(len(orig_table)):
#                 print(orig_table[i])
                if orig_table[i] != 0: 
                    new_table[ni] = orig_table[i]
                    ni+=1
            agent.groups_of_tables[curr_obs] = new_table
            
#             print(agent.groups_of_tables[curr_obs])

            # clean up TM
#             for d in delete_items:
# #                 if d >= np.shape(agent.TM)[0]: # MAYBE 
#                 agent.TM = np.delete(np.delete(agent.TM, d[2], axis=0), d[2], axis=1)

            # clean up COUNT
            for d in delete_items: 
                agent.count = np.delete(np.delete(agent.count, d[2], axis=0), d[2], axis=1)
                agent.TM = np.delete(np.delete(agent.TM, d[2], axis=0), d[2], axis=1)
                agent.C = np.delete(np.delete(agent.C, d[2], axis=0), d[2], axis=1)
            agent.normalize_TM()

            # clean up clone_map
#             print(clone_map)
#             for ids in delete_uniqueids: 
#                 print(ids)
#                 def clean_clone_map(data, delete_obs, delete_clone, delete_unique_id):
            clone_map = clean_clone_map(clone_map, delete_items)
#             print(clone_map)
            print(np.shape(agent.TM))


            log_lik = new_lik
    merge_lik = log_lik
    return agent, merge_lik



def learn_viterbi(agent, observations, actions, clone_map, clone_history,TM,n_iter=100):
    sys.stdout.flush()
    convergence = []
    pbar = trange(n_iter, position=0)
    log2_lik_old = -np.inf
    prior = get_distribution(agent.groups_of_tables)
    for it in pbar: 
        log2_lik, mess_fwd = forward_mp(agent, prior, observations, actions, clone_map, clone_history, TM, store_messages=True)
        states = backtrace(agent, observations, actions, mess_fwd, clone_map)
        agent.C[:] = 0
        for t in range(1, len(observations)):
            prev_action, prev_state, curr_state = (
                actions[t - 1],
                states[t - 1],
                states[t],
            )  # at time t-1 -> t we go from observation i to observation j
            agent.C[prev_state,curr_state,prev_action] += 1.0
        agent.normalize_TM()
#   def backtrace(agent, observations, actions, mess_fwd, clone_map):  

        convergence.append(-log2_lik.mean())
        pbar.set_postfix(train_bps=convergence[-1])
        if log2_lik.mean() <= log2_lik_old:
            break
        log2_lik_old = log2_lik.mean()

def plot_graph(
    agent, observations, actions, 
    clone_map, clone_history, output_file, cmap=cm.Spectral, multiple_episodes=False, vertex_size=30
):
    # pdb.set_trace()
#     states = chmm.decode(x, a)[1]
    _,states = decode(agent, observations, actions, clone_map, clone_history)

    v = np.unique(states)
#     print(v)
    if multiple_episodes:
        T = agent.C[:, v][:, :, v][:-1, 1:, 1:]
        v = v[1:]
    else:
        T = agent.C[v, :][:, v, :]
#     print(np.shape(T))
    A = T.sum(2)
#     print(A)
    A /= A.sum(1, keepdims=True)

    g = igraph.Graph.Adjacency((A > 0).tolist())
    node_labels = [clone[2] for clone in clone_map]
    node_labels = np.array(node_labels)
    
    # Node colors based on clone_map (or any observation data)
    node_colors = [t[0] for t in clone_map]  # Replace with your actual node color logic

    color_map = {0: 'pink', 1: 'skyblue', 2: 'green', 3: 'orange'}  # Observation to color mapping
    node_colors = [color_map[val] for val in node_colors]  # Flatten room to assign colors to each cell    
#     node_labels = np.arange(x.max() + 1).repeat(chmm.n_clones)[v]
    if multiple_episodes:
        node_labels -= 1
#     colors = [cmap(nl)[:3] for nl in node_labels / node_labels.max()]
    # out=[]
    out = igraph.plot(
        g,
        output_file,
        layout=g.layout("kamada_kawai"),
        vertex_color=node_colors,
        vertex_label=v,
        vertex_size=vertex_size,
        margin=50,
    )

    return out, v, g

In [ ]:
#2. Batch updating (recommended to begin with this)


# room = np.array(
#     [
#         [1, 2, 3, 0, 3, 1, 1, 1],
#         [1, 1, 3, 2, 3, 2, 3, 1],
#         [1, 1, 2, 0, 1, 2, 1, 0],
#         [0, 2, 1, 1, 3, 0, 0, 2],
#         [3, 3, 1, 0, 1, 0, 3, 0],
#         [2, 1, 2, 3, 3, 3, 2, 0],
#     ]
# )

room = np.array(
    [
        [1, 1, 1, 1, 3, 3, 3, 3],
        [1, 1, 1, 1, 3, 3, 3, 3],
        [1, 1, 1, 1, 3, 3, 3, 3],
        [2, 2, 2, 2, 0, 0, 0, 0],
        [2, 2, 2, 2, 0, 0, 0, 0],
        [2, 2, 2, 2, 0, 0, 0, 0],
    ]
)


actions, observations, rc = datagen_structured_obs_room(room, length=10000)     #Use length=50000 for bigger room
unique_obs = np.unique(observations)
pseudocount=1e-3
# np.random.seed(42)
agent = Agent(pseudocount=pseudocount)
alpha = 1
n_iter = 1 #100
n_particles = 10
n_actions = len(np.unique(actions))
n_obs = len(np.unique(observations))
scaling_factor = 3
pes = []
merge_lik = -np.inf
total_lls = []
total_indices = []
# indices = 0

for i in range(n_iter): 
    
    # 1. SPLITTING
    print('Splitting: iteration {}'.format(i))
    split_improved = False
#     agent_ = agent.copy()
#     agent_ = copy.deepcopy(agent)
    clone_history = []
    if i==0: # first iteration        
        clone_map= []   
        # initialize clones for each obs
#         clone_map.append((0,0,0))
#         clone_map.append((1,0,1))
#         clone_map.append((2,0,2))
#         clone_map.append((3,0,3))
        for uo in unique_obs:
            clone_map.append((uo,0,uo))
    
#         unique_clone_id =3
        unique_clone_id = len(unique_obs)-1
    else: 
        unique_clone_id = len(clone_map)-1
    lik_orig = 0
    for o, obs in enumerate(observations): 
        if o < 2:
            clone_history.append((obs,0, obs))
            # update transition
            if o ==1:
                agent.update_count(observations[o-1], observations[o], actions[o]) # update transition between t-1 and t-2
                agent.normalize_TM()                        
            continue # skip 1st observation
        splitted = False
        
        prev_prev_action = actions[o-2]
        prev_action = actions[o-1]
        prev_obs = observations[o-1]
        prev_tables = agent.count_tables_in_restaurant(prev_obs)
        # get the PE using the transition matrix
        
        prev_ind = [t[2] for t in clone_map if t[0]==prev_obs]
        # Get all state, actions that lead to current obs:
        obs_ind = [t[2] for t in clone_map if t[0]==obs] # this is list of curr obs clone id
        PE = 0
        for oi in obs_ind: # all possible clones related to current observation
            for pi in prev_ind: # all possible clones related to t-1 observation
            
                PE += agent.TM[pi, oi, prev_action] # just sum up PEs for now
#         PE = 
        pes.append(scaling_factor/PE)
        assignment, _ =  CRP(agent, prev_obs, alpha=scaling_factor/PE)
        
        post_tables = agent.count_tables_in_restaurant(prev_obs)
        if prev_tables != post_tables: # a new clone has been created for this observation

            splitted = True
            unique_clone_id += 1
            clone_map.append((prev_obs, assignment, unique_clone_id))
            agent.expand_split()
            prev_clone = unique_clone_id # clone id of t-1 : unique
            
        else:

            prev_clone = [t[2] for t in clone_map if t[0]==prev_obs and t[1]==assignment] # clone id of t-1 revealed
        prev_prev_obs, prev_prev_assignment, prev_prev_clone = clone_history[-1]
#         if i > 0: 
#             print(clone_history)


        clone_history.append((prev_obs, assignment, prev_clone)) # this keeps track of observation & clone history
    
        ################### CHANGING THE FW BW ONLINE
#         if o > 10: 
#             TM = agent.TM.copy()
#             prior = get_distribution(agent.groups_of_tables)

#             _, mess_fwd = forward_algorithm(agent, prior, observations[:o+1], actions[:o+1], clone_map, 
#                                             clone_history, TM, store_messages=True)
#             print(mess_fwd)
#             mess_bwd = backward_algorithm(agent, observations[:o+1], actions[:o+1], clone_map, TM)
#             print(mess_bwd)

#             agent.update_C(mess_fwd, mess_bwd, observations[:o+1], actions[:o+1], clone_map, clone_history)
        
        
    
        agent.update_count(prev_prev_clone, prev_clone, prev_prev_action) # update transition between t-1 and t-2
        agent.normalize_TM()
        ###################

    

    # 2. FORWARD ALGORITHM TO GET LIKELIHOOD
    TM = agent.TM.copy()
    prior = get_distribution(agent.groups_of_tables)
    split_lik_, mess_fwd = forward_algorithm(agent, prior, observations, actions, clone_map, clone_history, TM, store_messages=True)
    mess_bwd = backward_algorithm(agent, observations, actions, clone_map, TM)
    rev_mess_bwd = mess_bwd[::-1]
    forward_size = [len(f) for f in mess_fwd]
    backward_size = [len(b) for b in rev_mess_bwd]
#     print(forward_size)
#     print(backward_size)
    agent.update_C(mess_fwd, mess_bwd, observations, actions, clone_map, clone_history)
    agent.normalize_TM()
    
    
#     print(mess_bwd)
    #     print(len(mess_fwd))
    split_lik = split_lik_.mean()

    print('Log lik for split iteration {}: {}'.format(i, split_lik))
    print('TM size: {}'.format(np.shape(agent.TM)))
    
    total_lls.append(split_lik)
    total_indices.append(0)
    
    
#     draw_graph(agent, clone_map, niter=i)
#     output_file = 'sample.png'
#     graph, v, g = plot_graph(agent, observations, actions, 
#         clone_map, clone_history, output_file, cmap=cm.Spectral, multiple_episodes=False, vertex_size=30)
#     graph
# learn_viterbi(agent, observations, actions, clone_map, clone_history,TM,n_iter=100)
    name = 'split' + ' ' + str(i)
    draw_graph(agent, clone_map, niter=name)
#     if split_lik > merge_lik:
# #         agent = agent_
#         agent = copy.deepcopy(agent_)
        
#         print('split improved')
#         split_improved = True

    
            
    # 3. MERGING
#     pbar = trange(n_particles, position=0)
# if split_improved: 







#     log_lik = split_lik.copy()
#     for particle in range(n_particles): # many hypotheses // can this be parallelized?

#         curr_obs, clone_num, unique_idx = random.choice(clone_map[n_obs:])
#         shared_obs_clones = [t for t in clone_map[n_obs:] if t[0]==curr_obs and t[1] != clone_num] # all the clones that share the obs
#     #     prev_ind = [t[2] for t in clone_map if t[0]==prev_obs]
#     #     _, clone_id = clone_list[idx]
#         # do the merging by iterating over other clones
#         # in the sequence, find other clones that could be merged with the current

#         compare_clones = random.choice(shared_obs_clones) # pick a random clone that shares observation 
#         #### make this selection anti-proportional to number of assigned observations???

#         state_a = unique_idx
#         state_b = compare_clones[2]
# #         merged_TM = agent.TM.copy()
#         merged_count = agent.count.copy()

#         # adjusting prior distribution
#         merged_prior = get_distribution(agent.groups_of_tables)
#         merged_prior[state_a] += merged_prior[state_b]
#         merged_prior[state_b] = 0 #pseudocount    
#         # adjusting TM
#         for s in range(np.shape(agent.TM)[0]):
#             if s != state_a and s != state_b:
#                 for a in range(len(np.unique(actions))):
# #                     merged_TM[state_a][s][a] += merged_TM[state_b][s][a]
# #                     merged_TM[s][state_a][a] += merged_TM[s][state_b][a]
# #                     merged_TM[s][state_b][a] = pseudocount
# #                     merged_TM[state_b][s][a] = pseudocount
#                     merged_count[state_a][s][a] += merged_count[state_b][s][a]
#                     merged_count[s][state_a][a] += merged_count[s][state_b][a]
#                     merged_count[s][state_b][a] = pseudocount
#                     merged_count[state_b][s][a] = pseudocount
#         merged_TM = normalize_TM_count(merged_count)
#         # Remove state_b by zeroing out probabilities (optional)
#     #     agent.TM[state_b] = np.zeros(self.num_states)    

#         # Get the new prior
#     #     new_prior = # GET THE NEW DISTRIBUTION

#         # Get the new likelihood by forward algorithm
#     #     new_lik = new_prior * new_Tm

#         new_lik_, mess_fwd = forward_algorithm(agent, merged_prior, observations, actions, clone_map, clone_history, merged_TM)
# #         print(len(mess_fwd))
#         #     print(log_lik, new_lik)
#         new_lik = new_lik_.mean()
#         if new_lik > log_lik: # if merging is better: 
#             total_lls.append(new_lik)
#             total_indices.append(1)
#             agent.TM = merged_TM
#             agent.groups_of_tables[curr_obs][clone_num] += agent.groups_of_tables[curr_obs][compare_clones[1]]
#             agent.groups_of_tables[curr_obs][compare_clones[1]] = 0
#             print("Particle {}:".format(particle))
#             print("Merged Obs {} clone {} into clone {}, log lik: {}".format(curr_obs, compare_clones[1], clone_num, new_lik))
# #             pbar.set_postfix(train_bps=new_lik)

#             # clean up clones (including values that are 0)
# #             print(agent.groups_of_tables[curr_obs])
#             orig_table = agent.groups_of_tables[curr_obs].copy()
# #             print(orig_table)
#             orig_table[compare_clones[1]] = 0
#             delete_indices = [o for o,i in enumerate(orig_table) if orig_table[i]==0]
#             # Get unique IDs for the delete_indices from clone_map
#             delete_items = [
#                 t for t in clone_map
#                 for d in delete_indices
#                 if t[0] == curr_obs and t[1] == d
#             ]
# #                     delete_uniqueids = [
# #                 t[2] for t in clone_map
# #                 for d in delete_indices
# #                 if t[0] == curr_obs and t[1] == d
# #             ]
#             new_table = {}
#             ni = 0
#             for ii in range(len(orig_table)):
# #                 print(orig_table[i])
#                 if orig_table[ii] != 0: 
#                     new_table[ni] = orig_table[ii]
#                     ni+=1
#             agent.groups_of_tables[curr_obs] = new_table
            
# #             print(agent.groups_of_tables[curr_obs])

#             # clean up TM
# #             for d in delete_items:
# # #                 if d >= np.shape(agent.TM)[0]: # MAYBE 
# #                 agent.TM = np.delete(np.delete(agent.TM, d[2], axis=0), d[2], axis=1)

#             # clean up COUNT
#             for d in delete_items: 
#                 agent.count = np.delete(np.delete(agent.count, d[2], axis=0), d[2], axis=1)
#                 agent.TM = np.delete(np.delete(agent.TM, d[2], axis=0), d[2], axis=1)
#                 agent.C = np.delete(np.delete(agent.C, d[2], axis=0), d[2], axis=1)
#             agent.normalize_TM()

#             # clean up clone_map
# #             print(clone_map)
# #             for ids in delete_uniqueids: 
# #                 print(ids)
# #                 def clean_clone_map(data, delete_obs, delete_clone, delete_unique_id):
#             clone_map = clean_clone_map(clone_map, delete_items)
# #             print(clone_map)
#             print(np.shape(agent.TM))


#             log_lik = new_lik
#     merge_lik = log_lik











# #     print('\n')
# #     agent, merge_lik = merge(agent, observations, actions, split_lik, clone_map, clone_history, n_particles=10, )
#     name = 'merge '+ str(i)
#     draw_graph(agent, clone_map, niter=name)













    # ####### maybe instead of doing particle filter totally randomly, this can be rule-based too
    # ####### for example, proportional to clones with least number of tables will be more likely to be pruned
    # ####### probability for being selected == inverse of customers



    #         # merge according to number of particles, and update them if they improved

    # prev_prev_clone, prev_clone, prev_prev_action



In [ ]:
print(agent.TM[18,11,0])
print(agent.TM[18,11,1])
print(agent.TM[18,11,2])
print(agent.TM[18,11,3])



In [ ]:
TM_greedy

In [ ]:
draw_graph_TM(agent, clone_map, TM_greedy, niter=0, threshold=0, binary_edges=True)

In [ ]:
TM_greedy = np.zeros(np.shape(agent.TM))
for s_ in range(np.shape(agent.TM)[0]):
    for a_ in range(np.shape(agent.TM)[2]):
        
        maxtrans = -np.inf
        maxind = -np.inf
        for s in range(np.shape(agent.TM)[0]):
            if maxtrans < agent.TM[s_,s,a_]: 
                maxtrans = agent.TM[s_,s,a_]
                maxind = s
        # print(s_, maxtrans, maxind)
        TM_greedy[s_, maxind, a_] += 1
        print("action {} in state {} leads to {} with prob {}".format(a_, s_, maxind, maxtrans))
    print('\n')


In [ ]:
observations[:50]

In [ ]:
clone_history[:50]

In [ ]:
pes[:50]

In [ ]:
for s_ in range(5):
    for a_ in range(np.shape(agent.TM)[2]):
        
        maxtrans = -np.inf
        maxind = -np.inf
        for s in range(np.shape(agent.TM)[0]):
            # if maxtrans < agent.TM[s_,s,a_]: 
            #     maxtrans = agent.TM[s_,s,a_]
            #     maxind = s
        # print(s_, maxtrans, maxind)
            print("action {} in state {} leads to {} with prob {}".format(a_, s_,s, agent.TM[s_,s,a_]))
    print('\n')

In [ ]:
np.max(agent.TM[0,:,0
                ])

In [ ]:
np.shape(agent.TM)


In [ ]:
plt.figure(figsize=(12,6))
total_indices = np.array(total_indices)
split_indices = np.where(total_indices==0)[0]
merge_indices = np.where(total_indices==1)[0]
total_lls = -np.array(total_lls)
split_ll = [total_lls[i] for i in split_indices]
merge_ll = [total_lls[i] for i in merge_indices]
# plt.scatter()
# Plot splits
plt.scatter(split_indices, split_ll, color='red', label='Splits', s=100,zorder=2)

# Label each split a little above the dot
y_offset = 0.01  # Adjust this value as needed
for i, idx in enumerate(split_indices):
    plt.text(
        idx, 
        split_ll[i] + y_offset,  # Add offset to y-coordinate
        f"Split {i+1}", 
        fontsize=10, 
        color='red', 
        ha='center'
    )

# Plot merges
plt.scatter(merge_indices, merge_ll, color='blue', label='Merges', s=50, zorder=1)

plt.title("Training Curve: Splits and Merges", size=20)
# plt.xlabel("Iteration")
plt.ylabel("Negative log Likelihood", size=20)
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
np.shape(agent.TM)

In [ ]:
total_indices = np.array(total_indices)
split_indices = np.where(total_indices==0)[0]
split_indices

In [ ]:
merge_indices = np.where(total_indices==1)[0]
merge_indices

In [ ]:
total_indices

In [ ]:
# #2. Batch updating (recommended to begin with this)
# ############################################## don't touch this!###########################################

# room = np.array(
#     [
#         [1, 2, 3, 0, 3, 1, 1, 1],
#         [1, 1, 3, 2, 3, 2, 3, 1],
#         [1, 1, 2, 0, 1, 2, 1, 0],
#         [0, 2, 1, 1, 3, 0, 0, 2],
#         [3, 3, 1, 0, 1, 0, 3, 0],
#         [2, 1, 2, 3, 3, 3, 2, 0],
#     ]
# )
# actions, observations, rc = datagen_structured_obs_room(room, length=5000)     #Use length=50000 for bigger room
# pseudocount=1e-3
# np.random.seed(42)
# agent = Agent(pseudocount=pseudocount)
# alpha = 1
# n_iter = 1 #100
# n_particles = 50
# n_actions = len(np.unique(actions))
# n_obs = len(np.unique(observations))
# scaling_factor = 10
# pes = []
# for i in range(n_iter): 
    
#     # 1. SPLITTING
#     print('Splitting: iteration {}'.format(i))
#     print('TM size: {}'.format(np.shape(agent.TM)))
#     clone_history = []
#     if i==0: # first iteration        
#         clone_map= []   
#         # initialize clones for each obs
#         for obs in range(n_obs):
#             clone_map.append((obs,0,obs))    
#         unique_clone_id = n_obs-1
#     else: 
#         unique_clone_id = len(clone_map)-1
#     lik_orig = 0
#     for o, obs in enumerate(observations): 
#         if o < 2:
#             clone_history.append((obs,0, obs))
#             # update transition
#             if o ==1:
#                 agent.update_count(observations[o-1], observations[o], actions[o]) # update transition between t-1 and t-2
#                 agent.normalize_TM()                        
#             continue # skip 1st observation
#         splitted = False
        
#         prev_prev_action = actions[o-2]
#         prev_action = actions[o-1]
#         prev_obs = observations[o-1]
#         prev_tables = agent.count_tables_in_restaurant(prev_obs)
#         # get the PE using the transition matrix
        
#         prev_ind = [t[2] for t in clone_map if t[0]==prev_obs]
#         # Get all state, actions that lead to current obs:
#         obs_ind = [t[2] for t in clone_map if t[0]==obs] # this is list of curr obs clone id
#         PE = 0
#         for oi in obs_ind: # all possible clones related to current observation
#             for pi in prev_ind: # all possible clones related to t-1 observation
            
#                 PE += agent.TM[pi, oi, prev_action] # just sum up PEs for now
# #         PE = 
#         pes.append(PE)
#         assignment, _ =  CRP(agent, prev_obs, alpha=scaling_factor/PE)
        
#         post_tables = agent.count_tables_in_restaurant(prev_obs)
#         if prev_tables != post_tables: # a new clone has been created for this observation

#             splitted = True
#             unique_clone_id += 1
#             clone_map.append((prev_obs, assignment, unique_clone_id))
#             agent.expand_split_TM()
#             prev_clone = unique_clone_id # clone id of t-1 : unique
            
#         else:

#             prev_clone = [t[2] for t in clone_map if t[0]==obs and t[1]==assignment] # clone id of t-1 revealed
#         prev_prev_obs, prev_prev_assignment, prev_prev_clone = clone_history[-1]
# #         if i > 0: 
# #             print(clone_history)
#         agent.update_count(prev_prev_clone, prev_clone, prev_prev_action) # update transition between t-1 and t-2
#         agent.normalize_TM()

#         clone_history.append((prev_obs, assignment, prev_clone)) # this keeps track of observation & clone history

#     # 2. FORWARD ALGORITHM TO GET LIKELIHOOD
#     TM = agent.TM.copy()
#     prior = get_distribution(agent.groups_of_tables)
#     log_lik = forward_algorithm(agent, prior, observations, actions, clone_map, clone_history, TM)
# #     print(log_lik)
#     print('Log lik for split iteration {}: {}'.format(i, log_lik))
    
            
#     # 3. MERGING

#     for particle in range(n_particles): # many hypotheses // can this be parallelized?
#         print("Particle {}".format(particle))
#         curr_obs, clone_num, unique_idx = random.choice(clone_map[n_obs:])
#         shared_obs_clones = [t for t in clone_map[n_obs:] if t[0]==curr_obs and t[1] != clone_num] # all the clones that share the obs
#     #     prev_ind = [t[2] for t in clone_map if t[0]==prev_obs]
#     #     _, clone_id = clone_list[idx]
#         # do the merging by iterating over other clones
#         # in the sequence, find other clones that could be merged with the current

#         compare_clones = random.choice(shared_obs_clones) # pick a random clone that shares observation 
#         #### make this selection anti-proportional to number of assigned observations???

#         state_a = unique_idx
#         state_b = compare_clones[2]
#         merged_TM = agent.TM.copy()

#         # adjusting prior distribution
#         merged_prior = get_distribution(agent.groups_of_tables)
#         merged_prior[state_a] += merged_prior[state_b]
#         merged_prior[state_b] = 0 #pseudocount    
#         # adjusting TM
#         for s in range(np.shape(agent.TM)[0]):
#             if s != state_a and s != state_b:
#                 for a in range(len(np.unique(actions))):
#                     merged_TM[state_a][s][a] += merged_TM[state_b][s][a]
#                     merged_TM[s][state_a][a] += merged_TM[s][state_b][a]
#                     merged_TM[s][state_b][a] = pseudocount
#                     merged_TM[state_b][s][a] = pseudocount


#         # Remove state_b by zeroing out probabilities (optional)
#     #     agent.TM[state_b] = np.zeros(self.num_states)    

#         # Get the new prior
#     #     new_prior = # GET THE NEW DISTRIBUTION

#         # Get the new likelihood by forward algorithm
#     #     new_lik = new_prior * new_Tm
#         new_lik = forward_algorithm(agent, merged_prior, observations, actions, clone_map, clone_history, merged_TM)
#     #     print(log_lik, new_lik)
#         if new_lik > log_lik: # if merging is better: 
#             agent.TM = merged_TM
#             agent.groups_of_tables[curr_obs][clone_num] += agent.groups_of_tables[curr_obs][compare_clones[1]]
#             agent.groups_of_tables[curr_obs][compare_clones[1]] = 0
#             print("Merged Obs {} clone {} into clone {}, log lik: {}".format(curr_obs, compare_clones[1], clone_num, new_lik))
            
            
#             # clean up clones (including values that are 0)
#             print(agent.groups_of_tables[curr_obs])
#             orig_table = agent.groups_of_tables[curr_obs].copy()
# #             print(orig_table)
#             orig_table[compare_clones[1]] = 0
#             delete_indices = [o for o,i in enumerate(orig_table) if orig_table[i]==0]
#             # Get unique IDs for the delete_indices from clone_map
#             delete_items = [
#                 t for t in clone_map
#                 for d in delete_indices
#                 if t[0] == curr_obs and t[1] == d
#             ]
# #                     delete_uniqueids = [
# #                 t[2] for t in clone_map
# #                 for d in delete_indices
# #                 if t[0] == curr_obs and t[1] == d
# #             ]
#             new_table = {}
#             ni = 0
#             for i in range(len(orig_table)):
# #                 print(orig_table[i])
#                 if orig_table[i] != 0: 
#                     new_table[ni] = orig_table[i]
#                     ni+=1
#             agent.groups_of_tables[curr_obs] = new_table
# #             print(agent.groups_of_tables[curr_obs])
            
#             # clean up TM
#             for d in delete_items:
# #                 if d >= np.shape(agent.TM)[0]: # MAYBE 
#                 agent.TM = np.delete(np.delete(agent.TM, d[2], axis=0), d[2], axis=1)

#             # clean up COUNT
#             for d in delete_items: 
#                 agent.count = np.delete(np.delete(agent.count, d[2], axis=0), d[2], axis=1)     
                
#             # clean up clone_map
# #             print(clone_map)
# #             for ids in delete_uniqueids: 
# #                 print(ids)
# #                 def clean_clone_map(data, delete_obs, delete_clone, delete_unique_id):
#             clone_map = clean_clone_map(clone_map, delete_items)
# #             print(clone_map)
#             print(np.shape(agent.TM))
            
                        
#             log_lik = new_lik    
# # ####### maybe instead of doing particle filter totally randomly, this can be rule-based too
# # ####### for example, proportional to clones with least number of tables will be more likely to be pruned
# # ####### probability for being selected == inverse of customers
    
    
    
# #         # merge according to number of particles, and update them if they improved
                
# # prev_prev_clone, prev_clone, prev_prev_action



### Doing a more interactive process

In [ ]:
#2. Batch updating (recommended to begin with this)


room = np.array(
    [
        [1, 2, 3, 0, 3, 1, 1, 1],
        [1, 1, 3, 2, 3, 2, 3, 1],
        [1, 1, 2, 0, 1, 2, 1, 0],
        [0, 2, 1, 1, 3, 0, 0, 2],
        [3, 3, 1, 0, 1, 0, 3, 0],
        [2, 1, 2, 3, 3, 3, 2, 0],
    ]
)
actions, observations, rc = datagen_structured_obs_room(room, length=5000)     #Use length=50000 for bigger room
pseudocount=1e-3
np.random.seed(42)
# agent = Agent(pseudocount=pseudocount)
alpha = 1
n_iter = 2 #100
n_particles = 30
n_actions = len(np.unique(actions))
n_obs = len(np.unique(observations))
scaling_factor = 5
pes = []

agent = Agent(num_obs=n_obs, num_actions=n_actions,pseudocount=pseudocount)

for i in range(n_iter): 
    
    # 1. SPLITTING
    print('Splitting: iteration {}'.format(i))
    print('TM size: {}'.format(np.shape(agent.TM)))
    clone_history = []
    if i==0: # first iteration        
        clone_map= []   
        # initialize clones for each obs
        for obs in range(n_obs):
            clone_map.append((obs,0,obs))    
        unique_clone_id = n_obs-1
    else: 
        unique_clone_id = len(clone_map)-1
    lik_orig = 0
    for o, obs in enumerate(observations): 
        if o < 2:
            clone_history.append((obs,0, obs))
            # update transition
            if o ==1:
                agent.update_count(observations[o-1], observations[o], actions[o]) # update transition between t-1 and t-2
                agent.normalize_TM()                        
            continue # skip 1st observation
            
            
        splitted = False
        
        prev_prev_action = actions[o-2]
        prev_action = actions[o-1]
        prev_obs = observations[o-1]
        prev_tables = agent.count_tables_in_restaurant(prev_obs)
        # get the PE using the transition matrix
        
        prev_ind = [t[2] for t in clone_map if t[0]==prev_obs]
        # Get all state, actions that lead to current obs:
        obs_ind = [t[2] for t in clone_map if t[0]==obs] # this is list of curr obs clone id
        PE = 0
        for oi in obs_ind: # all possible clones related to current observation
            for pi in prev_ind: # all possible clones related to t-1 observation
            
                PE += agent.TM[pi, oi, prev_action] # just sum up PEs for now
#         PE = 
        pes.append(PE)
        assignment, _ =  CRP(agent, prev_obs, alpha=scaling_factor/PE)
        
        post_tables = agent.count_tables_in_restaurant(prev_obs)
        if prev_tables != post_tables: # a new clone has been created for this observation

            splitted = True
            unique_clone_id += 1
            clone_map.append((prev_obs, assignment, unique_clone_id))
            agent.expand_split_TM()
            prev_clone = unique_clone_id # clone id of t-1 : unique
            
        else:

            prev_clone = [t[2] for t in clone_map if t[0]==obs and t[1]==assignment] # clone id of t-1 revealed
        prev_prev_obs, prev_prev_assignment, prev_prev_clone = clone_history[-1]
#         if i > 0: 
#             print(clone_history)
        agent.update_count(prev_prev_clone, prev_clone, prev_prev_action) # update transition between t-1 and t-2
        agent.normalize_TM()

        clone_history.append((prev_obs, assignment, prev_clone)) # this keeps track of observation & clone history

    # 2. FORWARD ALGORITHM TO GET LIKELIHOOD
    TM = agent.TM.copy()
    prior = get_distribution(agent.groups_of_tables)
    log_lik = forward_algorithm(agent, prior, observations, actions, clone_map, clone_history, TM)
#     print(log_lik)
    print('Log lik for split iteration {}: {}'.format(i, log_lik))
    
            
    # 3. MERGING

    for particle in range(n_particles): # many hypotheses // can this be parallelized?
        print("Particle {}".format(particle))
        curr_obs, clone_num, unique_idx = random.choice(clone_map[n_obs:])
        shared_obs_clones = [t for t in clone_map[n_obs:] if t[0]==curr_obs and t[1] != clone_num] # all the clones that share the obs
    #     prev_ind = [t[2] for t in clone_map if t[0]==prev_obs]
    #     _, clone_id = clone_list[idx]
        # do the merging by iterating over other clones
        # in the sequence, find other clones that could be merged with the current
        
        compare_clones = random.choice(shared_obs_clones) # pick a random clone that shares observation 
#         ############ make this selection anti-proportional to number of assigned observations
#         distribution = [agent.groups_of_tables[curr_obs][v[1]] for v in shared_obs_clones]
#         distribution = np.array(distribution).astype(np.float64)
#         inverted_probs = 1 / distribution
#         inverted_probs[distribution == 0] = 0  # Handle zero probabilities if present
#         normalized_probs = inverted_probs / inverted_probs.sum()
#         # Randomly select an element based on probabilities
#         compare_clones = shared_obs_clones[np.random.choice(len(shared_obs_clones), p=normalized_probs)]
#         ################################

        state_a = unique_idx
        state_b = compare_clones[2]
        merged_TM = agent.TM.copy()

        # adjusting prior distribution
        merged_prior = get_distribution(agent.groups_of_tables)
        merged_prior[state_a] += merged_prior[state_b]
        merged_prior[state_b] = 0 #pseudocount    
        # adjusting TM
        for s in range(np.shape(agent.TM)[0]):
            if s != state_a and s != state_b:
                for a in range(len(np.unique(actions))):
                    merged_TM[state_a][s][a] += merged_TM[state_b][s][a]
                    merged_TM[s][state_a][a] += merged_TM[s][state_b][a]
                    merged_TM[s][state_b][a] = pseudocount
                    merged_TM[state_b][s][a] = pseudocount


        # Remove state_b by zeroing out probabilities (optional)
    #     agent.TM[state_b] = np.zeros(self.num_states)    

        # Get the new prior
    #     new_prior = # GET THE NEW DISTRIBUTION

        # Get the new likelihood by forward algorithm
    #     new_lik = new_prior * new_Tm
        merged_TM
        new_lik = forward_algorithm(agent, merged_prior, observations, actions, clone_map, clone_history, merged_TM)
    #     print(log_lik, new_lik)
        if new_lik > log_lik: # if merging is better: 
            agent.TM = merged_TM
            agent.groups_of_tables[curr_obs][clone_num] += agent.groups_of_tables[curr_obs][compare_clones[1]]
            agent.groups_of_tables[curr_obs][compare_clones[1]] = 0
            print("Merged Obs {} clone {} into clone {}, log lik: {}".format(curr_obs, compare_clones[1], clone_num, new_lik))
            
            
            # clean up clones (including values that are 0)
            print(agent.groups_of_tables[curr_obs])
            orig_table = agent.groups_of_tables[curr_obs].copy()
#             print(orig_table)
            orig_table[compare_clones[1]] = 0
            delete_indices = [o for o,i in enumerate(orig_table) if orig_table[i]==0]
            # Get unique IDs for the delete_indices from clone_map
            delete_items = [
                t for t in clone_map
                for d in delete_indices
                if t[0] == curr_obs and t[1] == d
            ]
#                     delete_uniqueids = [
#                 t[2] for t in clone_map
#                 for d in delete_indices
#                 if t[0] == curr_obs and t[1] == d
#             ]
            new_table = {}
            ni = 0
            for i in range(len(orig_table)):
#                 print(orig_table[i])
                if orig_table[i] != 0: 
                    new_table[ni] = orig_table[i]
                    ni+=1
            agent.groups_of_tables[curr_obs] = new_table
#             print(agent.groups_of_tables[curr_obs])
            
            # clean up TM
            for d in delete_items:
#                 if d >= np.shape(agent.TM)[0]: # MAYBE 
                agent.TM = np.delete(np.delete(agent.TM, d[2], axis=0), d[2], axis=1)

            # clean up COUNT
#             for d in delete_items: 
                agent.count = np.delete(np.delete(agent.count, d[2], axis=0), d[2], axis=1)     
            agent.normalize_TM()
                
            # clean up clone_map
#             print(clone_map)
#             for ids in delete_uniqueids: 
#                 print(ids)
#                 def clean_clone_map(data, delete_obs, delete_clone, delete_unique_id):
            clone_map = clean_clone_map(clone_map, delete_items)
#             print(clone_map)
            print(np.shape(agent.TM))
            
                        
            log_lik = new_lik    
# ####### maybe instead of doing particle filter totally randomly, this can be rule-based too
# ####### for example, proportional to clones with least number of tables will be more likely to be pruned
# ####### probability for being selected == inverse of customers
    
    
    
#         # merge according to number of particles, and update them if they improved
                
# prev_prev_clone, prev_clone, prev_prev_action



In [ ]:
normalized_probs

In [ ]:
shared_obs_clones

In [ ]:
shared_obs_clones

In [ ]:
agent.groups_of_tables[1]

In [ ]:
[v for v in range(len(agent.groups_of_tables[curr_obs]))]

In [ ]:
distribution = [agent.groups_of_tables[curr_obs][v[1]] for v in shared_obs_clones]
distribution = np.array(distribution).astype(np.float64)
distribution

In [ ]:
distribution /= distribution.sum()

In [ ]:
distribution

In [ ]:
sum(distribution)

In [ ]:
plt.plot(pes)

In [ ]:
pes

In [ ]:
plt.hist(10/np.array(pes))

In [ ]:
clone_history

In [ ]:
np.shape(TM)

In [ ]:

# Example transition matrix (state, state, action)
# transition_matrix = agent.TM
TM=viterbi_decoding(agent)
# TM = agent.TM
num_states = np.shape(TM)[0]
num_actions = np.shape(TM)[2]

# Threshold to show only significant transitions
threshold = 0.9

# Create a directed graph
G = nx.DiGraph()

# Add nodes for each state
for state in range(num_states):
    G.add_node(state)

# Add edges based on the transition matrix
for action in range(num_actions):
    for from_state in range(num_states):
        for to_state in range(num_states):
            weight = TM[from_state, to_state, action]
            if weight > threshold:  # Filter out weak transitions
                G.add_edge(
                    from_state,
                    to_state,
                    weight=weight,
                    label=f"A{action}"  # Label edges by action
                )

# Use Kamada-Kawai layout for a more spatially balanced layout
pos = nx.kamada_kawai_layout(G)  # Alternative layouts: spring_layout, shell_layout

# Node colors based on clone_map (or any observation data)
node_colors = [t[0] for t in clone_map]  # Replace with your actual node color logic

color_map = {0: 'pink', 1: 'skyblue', 2: 'green', 3: 'orange'}  # Observation to color mapping
node_colors = [color_map[val] for val in node_colors]  # Flatten room to assign colors to each cell

# Draw the graph
plt.figure(figsize=(12, 8))
nx.draw(
    G,
    pos,
    with_labels=True,
    node_color=node_colors,  # Use the color scheme for nodes
    node_size=1000,
    font_size=10,
    edge_color='gray',
    arrowsize=20
)

# Draw edge labels
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(
    G,
    pos,
    edge_labels=edge_labels,
    font_size=8
)

plt.title("Spatial Transition Graph")
plt.axis('off')  # Turn off axes for cleaner visualization
plt.show()


In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# Example transition matrix (state, state, action)
# transition_matrix = agent.TM
TM=viterbi_decoding(agent)
# TM = agent.TM
num_states = np.shape(TM)[0]
num_actions = np.shape(TM)[2]

# Threshold to show only significant transitions
threshold = 0.2

# Create a directed graph
G = nx.DiGraph()

# Add nodes for each state
for state in range(num_states):
    G.add_node(state)

# Add edges based on the transition matrix
for action in range(num_actions):
    for from_state in range(num_states):
        for to_state in range(num_states):
            weight = TM[from_state, to_state, action]
            if weight > threshold:  # Filter out weak transitions
                G.add_edge(
                    from_state,
                    to_state,
                    weight=weight,
                    label=f"A{action}"  # Label edges by action
                )

# Use Kamada-Kawai layout for a more spatially balanced layout
pos = nx.kamada_kawai_layout(G)  # Alternative layouts: spring_layout, shell_layout

# Node colors based on clone_map (or any observation data)
node_colors = [t[0] for t in clone_map]  # Replace with your actual node color logic

color_map = {0: 'pink', 1: 'skyblue', 2: 'green', 3: 'orange'}  # Observation to color mapping
node_colors = [color_map[val] for val in node_colors]  # Flatten room to assign colors to each cell

# Draw the graph
plt.figure(figsize=(12, 8))
nx.draw(
    G,
    pos,
    with_labels=True,
    node_color=node_colors,  # Use the color scheme for nodes
    node_size=1000,
    font_size=10,
    edge_color='gray',
    arrowsize=20
)

# Draw edge labels
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(
    G,
    pos,
    edge_labels=edge_labels,
    font_size=8
)

plt.title("Spatial Transition Graph")
plt.axis('off')  # Turn off axes for cleaner visualization
plt.show()
# 

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# Example transition matrix (state, state, action)
# transition_matrix = agent.TM
TM=viterbi_decoding(agent)
# TM = agent.TM
num_states = np.shape(TM)[0]
num_actions = np.shape(TM)[2]

# Threshold to show only significant transitions
threshold = 0.2

# Create a directed graph
G = nx.DiGraph()

# Add nodes for each state
for state in range(num_states):
    G.add_node(state)

# Add edges based on the transition matrix
for action in range(num_actions):
    for from_state in range(num_states):
        for to_state in range(num_states):
            weight = TM[from_state, to_state, action]
            if weight > threshold:  # Filter out weak transitions
                G.add_edge(
                    from_state,
                    to_state,
                    weight=weight,
                    label=f"A{action}"  # Label edges by action
                )

# Use Kamada-Kawai layout for a more spatially balanced layout
pos = nx.kamada_kawai_layout(G)  # Alternative layouts: spring_layout, shell_layout

# Node colors based on clone_map (or any observation data)
node_colors = [t[0] for t in clone_map]  # Replace with your actual node color logic

color_map = {0: 'pink', 1: 'skyblue', 2: 'green', 3: 'orange'}  # Observation to color mapping
node_colors = [color_map[val] for val in node_colors]  # Flatten room to assign colors to each cell

# Draw the graph
plt.figure(figsize=(12, 8))
nx.draw(
    G,
    pos,
    with_labels=True,
    node_color=node_colors,  # Use the color scheme for nodes
    node_size=1000,
    font_size=10,
    edge_color='gray',
    arrowsize=20
)

# Draw edge labels
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(
    G,
    pos,
    edge_labels=edge_labels,
    font_size=8
)

plt.title("Spatial Transition Graph")
plt.axis('off')  # Turn off axes for cleaner visualization
plt.show()
# 

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# Example transition matrix (state, state, action)
# transition_matrix = agent.TM
TM=viterbi_decoding(agent)
# TM = agent.TM
num_states = np.shape(TM)[0]
num_actions = np.shape(TM)[2]

# Threshold to show only significant transitions
threshold = 0.1

# Create a directed graph
G = nx.DiGraph()

# Add nodes for each state
for state in range(num_states):
    G.add_node(state)

# Add edges based on the transition matrix
for action in range(num_actions):
    for from_state in range(num_states):
        for to_state in range(num_states):
            weight = TM[from_state, to_state, action]
            if weight > threshold:  # Filter out weak transitions
                G.add_edge(
                    from_state,
                    to_state,
                    weight=weight,
                    label=f"A{action}"  # Label edges by action
                )

# Use Kamada-Kawai layout for a more spatially balanced layout
pos = nx.kamada_kawai_layout(G)  # Alternative layouts: spring_layout, shell_layout

# Node colors based on clone_map (or any observation data)
node_colors = [t[0] for t in clone_map]  # Replace with your actual node color logic

color_map = {0: 'pink', 1: 'skyblue', 2: 'green', 3: 'orange'}  # Observation to color mapping
node_colors = [color_map[val] for val in node_colors]  # Flatten room to assign colors to each cell

# Draw the graph
plt.figure(figsize=(12, 8))
nx.draw(
    G,
    pos,
    with_labels=True,
    node_color=node_colors,  # Use the color scheme for nodes
    node_size=1000,
    font_size=10,
    edge_color='gray',
    arrowsize=20
)

# Draw edge labels
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(
    G,
    pos,
    edge_labels=edge_labels,
    font_size=8
)

plt.title("Spatial Transition Graph")
plt.axis('off')  # Turn off axes for cleaner visualization
plt.show()
# 

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# Example transition matrix (state, state, action)
# transition_matrix = agent.TM
TM=viterbi_decoding(agent)
# TM = agent.TM
num_states = np.shape(TM)[0]
num_actions = np.shape(TM)[2]

# Threshold to show only significant transitions
threshold = 0.2

# Create a directed graph
G = nx.DiGraph()

# Add nodes for each state
for state in range(num_states):
    G.add_node(state)

# Add edges based on the transition matrix
for action in range(num_actions):
    for from_state in range(num_states):
        for to_state in range(num_states):
            weight = TM[from_state, to_state, action]
            if weight > threshold:  # Filter out weak transitions
                G.add_edge(
                    from_state,
                    to_state,
                    weight=weight,
                    label=f"A{action}"  # Label edges by action
                )

# Use Kamada-Kawai layout for a more spatially balanced layout
pos = nx.kamada_kawai_layout(G)  # Alternative layouts: spring_layout, shell_layout

# Node colors based on clone_map (or any observation data)
node_colors = [t[0] for t in clone_map]  # Replace with your actual node color logic

color_map = {0: 'pink', 1: 'skyblue', 2: 'green', 3: 'orange'}  # Observation to color mapping
node_colors = [color_map[val] for val in node_colors]  # Flatten room to assign colors to each cell

# Draw the graph
plt.figure(figsize=(12, 8))
nx.draw(
    G,
    pos,
    with_labels=True,
    node_color=node_colors,  # Use the color scheme for nodes
    node_size=1000,
    font_size=10,
    edge_color='gray',
    arrowsize=20
)

# Draw edge labels
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(
    G,
    pos,
    edge_labels=edge_labels,
    font_size=8
)

plt.title("Spatial Transition Graph")
plt.axis('off')  # Turn off axes for cleaner visualization
plt.show()
# 

In [ ]:
# action from one state to other should be more deterministic?
# draw this graph thresholded 
# like viterbi decoding, only draw the highest prob. action (1 or 0) -- binarize the TM

In [ ]:
agent.groups_of_tables

In [ ]:
for t in range(4):
    print(len(agent.groups_of_tables[t]))

In [ ]:
# Count unique elements and their frequencies
np.unique(room, return_counts=True)

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# Example transition matrix (state, state, action)
transition_matrix = agent.TM
num_states = np.shape(agent.TM)[0]
num_actions = np.shape(agent.TM)[2]
# transition_matrix = np.random.rand(num_states, num_states, num_actions)

# Threshold to show only significant transitions
threshold = 0.1

# Example observations for each state (one of four categories)
# Replace with your actual observations
# observations = [0, 1, 2, 3, 1]  # Each state is assigned to an observation
# color_map = {0: 'red', 1: 'blue', 2: 'green', 3: 'orange'}  # Observation to color mapping

# Assign colors to states based on observations
# node_colors = [color_map[obs] for obs in observations]
node_colors = [t[0] for t in clone_map]
# Create a directed graph
G = nx.DiGraph()

# Add nodes for each state
for state in range(num_states):
    G.add_node(state)

# Add edges based on the transition matrix
for action in range(num_actions):
    for from_state in range(num_states):
        for to_state in range(num_states):
            weight = transition_matrix[from_state, to_state, action]
            if weight > threshold:  # Filter out weak transitions
                G.add_edge(
                    from_state,
                    to_state,
                    weight=weight,
                    label=f"A{action}"  # Label edges by action
                )

# Draw the graph
pos = nx.spring_layout(G)  # Position nodes using a spring layout
edge_labels = nx.get_edge_attributes(G, 'label')  # Get edge labels

plt.figure(figsize=(10, 8))
nx.draw(
    G,
    pos,
    with_labels=True,
    node_color=node_colors,  # Use the color scheme for nodes
    node_size=1000,
    font_size=10,
    edge_color='gray',
    arrowsize=20
)
nx.draw_networkx_edge_labels(
    G,
    pos,
    edge_labels=edge_labels,
    font_size=8
)

# Add a legend for the color scheme
legend_labels = [f"Observation {obs}: {color}" for obs, color in color_map.items()]
legend_text = "\n".join(legend_labels)
plt.gcf().text(0.9, 0.5, legend_text, fontsize=10, va='center')

plt.title("Transition Graph with Observations")
plt.show()


In [ ]:
import seaborn as sns
sns.heatmap(agent.TM[:][0][:])

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# Ground-truth environment
room = np.array(
    [
        [1, 2, 3, 0, 3, 1, 1, 1],
        [1, 1, 3, 2, 3, 2, 3, 1],
        [1, 1, 2, 0, 1, 2, 1, 0],
        [0, 2, 1, 1, 3, 0, 0, 2],
        [3, 3, 1, 0, 1, 0, 3, 0],
        [2, 1, 2, 3, 3, 3, 2, 0],
    ]
)

# Define the color map for observations
color_map = {0: 'pink', 1: 'skyblue', 2: 'green', 3: 'orange'}  # Observation to color mapping
node_colors = [color_map[val] for val in room.flatten()]  # Flatten room to assign colors to each cell


# Create a graph
rows, cols = room.shape
GT = nx.Graph()

# Add nodes
for r in range(rows):
    for c in range(cols):
        node_id = (r, c)
        GT.add_node(node_id, observation=room[r, c])

# Add edges for neighbors (up, down, left, right)
for r in range(rows):
    for c in range(cols):
        if r > 0:  # Connect to the cell above
            GT.add_edge((r, c), (r-1, c))
        if r < rows - 1:  # Connect to the cell below
            GT.add_edge((r, c), (r+1, c))
        if c > 0:  # Connect to the cell to the left
            GT.add_edge((r, c), (r, c-1))
        if c < cols - 1:  # Connect to the cell to the right
            GT.add_edge((r, c), (r, c+1))

# Position nodes in a grid layout
pos = {(r, c): (c, -r) for r in range(rows) for c in range(cols)}  # Flip y-axis for grid alignment

# Draw the graph
plt.figure(figsize=(12, 8))
nx.draw(
    GT,
    pos,
    with_labels=True,
    labels={(r, c): f"{room[r, c]}" for r in range(rows) for c in range(cols)},  # Show observation value
    node_color=node_colors,  # Use colors based on observations
    node_size=500,
    font_size=8,
    edge_color='gray'
)

# Add a legend
legend_labels = [f"Observation {obs}: {color}" for obs, color in color_map.items()]
legend_patches = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10)
                  for color in color_map.values()]
plt.legend(legend_patches, legend_labels, loc='upper left', title="Observation Key")

plt.title("Graph Representation of Spatial Environment")
plt.axis('off')
plt.show()


In [ ]:
room

In [ ]:
############# Before putting it to agent.forward_algorithm #############    
#     prior = get_distribution(agent.groups_of_tables)
#     print(prior)

#     num_steps = len(observations)
#     num_states = len(prior)
# #     forward_probs = np.zeros((num_states, num_steps)) # n total clone, sequence length
#     log_forward_probs = np.full((num_states, num_steps), 0)  # Use -inf for log(0)

#     initial_state = observations[0]
#     for s in range(num_states):
#         if s == initial_state:
# #             forward_probs[s, 0] = prior[s]
#             log_forward_probs[s,0] = prior[s]
#         else:
# #             forward_probs[s, 0] = 0  # Probability is zero if it doesn't match initial state
#             log_forward_probs[s,0] = 0


#     for t in range(1, num_steps):
#         current_clone = [clone[2] for clone in clone_map 
#                          if clone[0]==clone_history[t][0] and clone[1]==clone_history[t][1]]

#         prev_action = actions[t-1]

#         for s in range(num_states):
#                 log_forward_probs[s, t] = np.logaddexp.reduce([
#                     log_forward_probs[prev_s, t-1] + np.log(agent.TM[prev_s, s, prev_action])
#                     for prev_s in range(num_states)
#                 ])
#     print(np.sum(log_forward_probs))
############################################################################## 

In [ ]:
agent.groups_of_tables

In [ ]:
print(clone_map)

In [ ]:
clone_map

In [ ]:
np.sum(forward_probs)

In [ ]:
observations[0]

In [ ]:
agent.get_total_observations()

In [ ]:
for i in range(4):
#     curr_
    print(agent.get_restaurant_total_customers(i))

In [ ]:
[agent.get_restaurant_total_customers(i)/agent.get_total_observations() for i in range(4)]

In [ ]:
agent.get_prior_distribution()

In [ ]:
for i in range(4): 
    print(agent.count_tables_in_restaurant(i))

# Problems to consider: 
1. TM size is +4 the number of clones
2. sum of probs (PE) doesn't necessarily have to be less than 1 but seems so. Why?

In [ ]:
clone_history

In [ ]:
data= agent.groups_of_tables

In [ ]:
def get_distribution(data):
    """
    Compute the probability distribution from a nested dictionary structure.

    Parameters:
        data (dict): A nested dictionary where the outer keys map to inner dictionaries
                     containing counts for different elements.

    Returns:
        np.ndarray: A NumPy array containing the probability distribution.
    """
    # Flatten the dictionary and compute total count
    flattened_counts = defaultdict(int)
    total_count = 0

    for outer_key, inner_dict in data.items():
        for inner_key, count in inner_dict.items():
            flattened_counts[inner_key] += count
            total_count += count

    # Compute probabilities as a sorted NumPy array
    sorted_keys = sorted(flattened_counts.keys())
    probabilities = np.array([flattened_counts[key] / total_count for key in sorted_keys])

    return probabilities


In [ ]:
get_distribution(data)


In [ ]:
np.shape(agent.TM)

In [ ]:
clone_history

### General intuition
At first, many clones will be splitted out due to vanilla transition matrix

But this will be corrected with merging

With more experience and more accurate transition matrices, splitting and merging will diminish

we can end whenever the likelihood plataeus. 


In [ ]:
import random
random.sample([1,2],1)

In [ ]:
container.groups_of_tables

In [ ]:
n_clones

In [ ]:
PEs = np.array([np.sum(T_tr[a[t-1], old_ind, state_loc[i]:state_loc[i+1]]) for i in range(len(state_loc)-1)])
if len(state_loc) < 5: ####CHECK THIS PART LATER
    PEs = np.append(PEs, np.sum(T_tr[a[t-1], old_ind, state_loc[-1]:]))
PEs_softmax = softmax(PEs)
PE = PEs_softmax[ind]

In [ ]:
n_clones

In [ ]:
container.get_group_total(0)

In [ ]:
container.get_total_observations()

In [ ]:
class splitMerge: 
    def __init__(self, alpha=1.0, depth=3):
    # splitting
    self.alpha = alpha   # Dispersion parameter for CRP
    self.depth = depth   # Maximum depth of the tree
    self.tree = defaultdict(lambda: defaultdict(int))  # Tracks customer counts at each node
    # merging
#         def __init__(self, num_states, transition_probs, emission_probs, alpha=1.0):
    self.num_states = num_states
    self.transition_probs = transition_probs
    self.emission_probs = emission_probs
    self.alpha = alpha  # Dirichlet prior